In [1]:
using ReverseDiffSource


In [2]:
# Prob 6, dimension = 2, x* = [1,1]
f6(x) = (1 - x[1])^2
c6(x) = 10*(x[2] - x[1]^2)
x6_0 = [-1.2, 1]

# Prob 7, dimension = 2, x* = [0,sqrt(3)]
f7(x) = log(1 + x[1]^2) - x[2]
c7(x) = (1 + x[1]^2)^2 + x[2]^2 - 4
x7_0 = [2, 2]

# Prob 8, dimension = 2, x* = [a,9/a], [-a,-9/a], a = sqrt( ( 25 +- sqrt(301) )/2 )
f8(x) = -1*x[1]^0
c81(x) = x[1]^2 + x[2]^2 - 25
c82(x) = x[1]*x[2] - 9
x8_0 = [2,1]

# Prob 9, dimension = 2, x* = [12k - 3, 16k - 4] for integer k
f9(x) = sin(pi*x[1]/12)*cos(pi*x[2]/16)
c9(x) = 4*x[1] - 3*x[2]
x9_0 = [0,0]

2-element Array{Int64,1}:
 0
 0

In [3]:
#x[3] = y, x[4] = rho
L6(x) = (1 - x[1])^2 - x[3]*10*(x[2] - x[1]^2) + (x[4]/2)*(10*(x[2] - x[1]^2))^2
L7(x) = log(1 + x[1]^2) - x[2] - x[3]*((1 + x[1]^2)^2 + x[2]^2 - 4) + (x[4]/2)*((1 + x[1]^2)^2 + x[2]^2 - 4)^2
L8(x) = -1 - x[3]*(x[1]^2 + x[2]^2 - 25) - x[4]*(x[1]*x[2] - 9) + (x[5]/2)*((x[1]^2 + x[2]^2 - 25)^2 + (x[1]*x[2] - 9)^2)
L9(x) = sin(pi*x[1]/12)*cos(π*x[2]/16) - x[3]*(4*x[1] - 3*x[2]) + (x[4]/2)*(4*x[1] - 3*x[2])^2


diff_f6 = rdiff( f6, (ones(2),), order = 2)
diff_f7 = rdiff( f7, (ones(2),), order = 2)
diff_f8 = rdiff( f8, (ones(1),), order = 2)
diff_f9 = rdiff( f9, (ones(2),), order = 2)

diff_c6 = [rdiff( c6, (ones(2),), order = 2)];
diff_c7 = [rdiff( c7, (ones(2),), order = 2)];
diff_c81 = rdiff( c81, (ones(2),), order = 2)
diff_c82 = rdiff( c82, (ones(2),), order = 2)
#diff_c8 = [diff_c81, diff_c82];
diff_c9 = [rdiff( c9, (ones(2),), order = 2)];

diff_L6 = rdiff( L6, (ones(4),), order = 2)
diff_L7 = rdiff( L7, (ones(4),), order = 2)
diff_L8 = rdiff( L8, (ones(5),), order = 2)
diff_L9 = rdiff( L9, (ones(4),), order = 2)

#function c(v,x)
#    m = size(v,1);
#    eval = [];
#    grad = [];
#    for i=1:m
#        eval = [eval, v[i](x)(1)];
#        grad = [grad; v[i](x)(2)]
#    end
#    return [eval,grad];
#end


newdiff_L6 = (function (x,y,rho)
    tmp=diff_L6([x,y,rho]);
    return (tmp[1], tmp[2][1:2],tmp[3][1:2,1:2])
end)

newdiff_L7 = (function (x,y,rho)
    tmp=diff_L7([x,y,rho]);
    return (tmp[1], tmp[2][1:2],tmp[3][1:2,1:2])
end)

#newdiff_L8 = (function (x,y,rho)
#    tmp=diff_L8([x,y,rho]);
#    return (tmp[1], tmp[2][1:2],tmp[3][1:2,1:2])
#end)

newdiff_L9 = (function (x,y,rho)
    tmp=diff_L9([x,y,rho]);
    return (tmp[1], tmp[2][1:2],tmp[3][1:2,1:2])
end)


(anonymous function)

In [4]:
function warmup(warmUp, obj, x, its, fvals, gNormVals)
    for i = 1:warmUp
        (f,g,H) = obj(x);
        (sigma, V) = eig(H)
        L = maximum(sigma)
        x = x - (1/L)*g

        its = its + 1;
        fvals = [fvals; f];
        gNormVals = [gNormVals; norm(g,2)];
    end
    return (x,its,fvals,gNormVals)
end

warmup (generic function with 1 method)

In [5]:
function newtstep1(obj,x,its,fvals,gNormVals)
    (f,g,H) = obj(x);
        
    #modify H if not pos def
    E = eigfact(H);
    V = E[:vectors];
    lambda = diagm(max(abs(E[:values]),1e-2));
    d = -V*inv(lambda)*transpose(V)*g;
        
    #backtracking linesearch
    alpha = 1;
    mu = 10^-2.0;
    (newf,newg,newH) = obj(x+alpha*d);
    while newf > f + (alpha*mu)*(dot(g,d))
        (newf, newg, newH) = obj(x + alpha*d);
        alpha = alpha/2;
    end

    x = x+alpha*d;

    its = its + 1;
    fvals = [fvals; f];
    gNormVals= [gNormVals; norm(g,2)];
    return(x,g,its,fvals,gNormVals);
end


newtstep1 (generic function with 1 method)

In [6]:
function newtstep2(obj,x,its,fvals,gNormVals)
    (f,g,H) = obj(x);
        
    #modify H if not pos def
    E = eigfact(H);
    V = E[:vectors];
    lambda = diagm(max(E[:values],1e-2));
    d = -V*inv(lambda)*transpose(V)*g;
        
    #backtracking linesearch
    alpha = 1;
    mu = 10^-2.0;
    (newf,newg,newH) = obj(x+alpha*d);
    while newf > f + (alpha*mu)*(dot(g,d))
        (newf, newg, newH) = obj(x + alpha*d);
        alpha = alpha/2;
    end

    x = x+alpha*d;

    its = its + 1;
    fvals = [fvals; f];
    gNormVals= [gNormVals; norm(g,2)];
    return(x,g,its,fvals,gNormVals);
end


newtstep2 (generic function with 1 method)

In [7]:
function BFGSstep(obj,x,B,its,fvals,gNormVals)
    (f,g,H) = obj(x);
    d = -B\g;
        
    
    #backtracking linesearch with Wolfe condition
    alpha = 1;
    mu = 10^-2.0;
    nu = 0.8;
    oldx = x;
    (oldf,oldg,oldH) = obj(x);
    (newf,newg,newH) = obj(x+alpha*d);
    alphaDir = 0;
    while newf > f + (alpha*mu)*(dot(g,d))
        (newf, newg, newH) = obj(x + alpha*d);
        alpha = alpha/2;
    end

    x = x+alpha*d;
    (f,g,H) = obj(x);
    q = alpha*B*d;
    y = g - oldg;
    s = x - oldx;
    
    B = B - (1/(transpose(q)*s)[1])*(q*transpose(q)) + (1/(transpose(y)*s)[1])*(y*transpose(y));
    
    its = its + 1;
    fvals = [fvals; f];
    gNormVals= [gNormVals; norm(g,2)];
    return(x,B,g,its,fvals,gNormVals);
end

BFGSstep (generic function with 1 method)

In [8]:
function Or(boolArray)
    for i=1:size(boolArray,1)
        if boolArray[i] == true
            return (true, i)
        end
    end
    return (false,size(boolArray,1))
end

Or (generic function with 1 method)

In [9]:
function newtmin(obj, x0, maxIts, optTol, warmUp, whileSwitch, randPerts,newtVers)
    fvals = [];
    gNormVals=[];
    (f0,g0,H0) = obj(x0);
    x = x0;
    (f,g,H) = obj(x);
    its = 0;
    B = eye(size(x0,1));
    

    if warmUp > 0
        (x,its,fvals,gNormVals) = warmup(warmUp,obj,x,its,fvals,gNormVals);
    end
    
    while (norm(g,2) > optTol*whileSwitch*norm(g0,2) && norm(g,2) > 1e-8 && its < maxIts)
        if newtVers == 1
            (x,g,its,fvals,gNormVals) = newtstep1(obj,x,its,fvals,gNormVals);
        end
        if newtVers == 2
            (x,g,its,fvals,gNormVals) = newtstep2(obj,x,its,fvals,gNormVals);
        end
        if newtVers == 3
            (x,B,g,its,fvals,gNormVals) = BFGSstep(obj,x,B,its,fvals,gNormVals)
        end
    end 
    
    if norm(g,2) < optTol*whileSwitch*norm(g0,2) || norm(g,2) < 1e-8
#        println("Method computed optimal x: ", x)
#        println("\nIterations: ", its)
#        println("\nObjective: ", f)
#        println("\nInitial gradient norm: ", norm(g0,2))
#        println("\nGradient norm: ", norm(g,2))
        return (x, its, fvals, gNormVals)
    else
        perts = 0;
        while perts < randPerts
            E = eigfact(H);
            lambda = E[:values];
            V = E[:vectors];
            if Or(lambda.<0)[1]
                perts = perts + 1
                x = x + optTol*V[Or(lambda.<0)[2]];         #use small perturbation in decreasing eigenvector direction
                while (norm(g,2) > optTol*whileSwitch*norm(g0,2) && norm(g,2) > 1e-8 && its < (perts + 1)*maxIts)
                    (x,g,its,fvals,gNormVals) = newtstep1(x,its,fvals,gNormVals);
                end 
            end
            if norm(g,2) < optTol*whileSwitch*norm(g0,2) || norm(g,2) < 1e-8
#                println("Method computed optimal x: ", x)
#                println("\nIterations: ", its)
#                println("\nObjective: ", f)
#                println("\nInitial gradient norm: ", norm(g0,2))
#                println("\nGradient norm: ", norm(g,2))
                return (x, its, fvals, gNormVals)
            end
        end
#        println("Method timed out on maxIts: ", its)
#        println("\nx: ", x)
#        println("\nObjective: ", f)
#        println("\nInitial gradient norm: ", norm(g0,2))
#        println("\nGradient norm: ", norm(g,2))
    end
    return (x, its, fvals, gNormVals);
end

newtmin (generic function with 1 method)

In [38]:
function augLag(LagOracle,fcnOracle,constraintOracle,x0,maxIts,newtvers)
    f = fcnOracle
    c = constraintOracle[1]
    x = x0

    k = 1
    eta = 1/k
    rho = 1e-2
    cTol = 1e-6
    gTol = 1e-6           

    y = -rho*((c(x))[1])
    L(x) = LagOracle(x,y,rho)
    while ((norm(c(x)[1]) > cTol) || (norm(f(x)[2] - c(x)[2]*y) > gTol) ) && (k < maxIts)
        if (norm(c(x)[1]) < eta)
            y = y - rho*c(x)[1]
        else
            rho = 10*rho
            L(x) = LagOracle(x,y,rho)
        end
        (x, its, fvals, gNormVals) = newtmin(L, x, 1000, 1e-3, 10, 1, 0, newtvers)
        k = k + 1
        eta = 1/k
        println("passed iteration ",k," with x =",x,", c(x) = ",c(x)[1])
    end
    return (x, k)
end


augLag (generic function with 2 methods)

In [39]:
(xstar,k)=augLag(newdiff_L6, diff_f6, diff_c6, x6_0, 100,3)
println("Aug Lag with BFGS produced: ", xstar, " in ", k , " iterations.");
(xstar,k)=augLag(newdiff_L6, diff_f6, diff_c6, x6_0, 100,1)
println("Aug Lag with exact Newton produced: ", xstar, " in ", k, "iterations.");

passed iteration 2 with x =[0.9909584015022326,1.0252183517762168], c(x) = 0.43219798268356757
passed iteration 3 with x =[1.0000038301809457,1.0007875817604626], c(x) = 0.007799213839008168
passed iteration 4 with x =[0.9999997826405542,1.0000001156126577], c(x) = 5.503315019250721e-6
passed iteration 5 with x =[1.0000000000005478,0.9999997300159585], c(x) = -2.699851370246975e-6
passed iteration 6 with x =[1.0000000000002591,1.0000000000012355], c(x) = 7.172040739078511e-12
Aug Lag with BFGS produced: [1.0000000000002591,1.0000000000012355] in 6 iterations.
passed iteration 2 with x =[0.9999993580066834,1.043998697340156], c(x) = 0.4399998132637717
passed iteration 3 with x =[0.9999999999997822,1.00000001867291], c(x) = 1.8673345758557502e-7
Aug Lag with exact Newton produced: [0.9999999999997822,1.00000001867291] in 3iterations.


In [40]:
(xstar,k)=augLag(newdiff_L7, diff_f7, diff_c7, x7_0, 100,3)
println("Aug Lag with BFGS produced: ", xstar, " in ", k , " iterations.");
(xstar,k)=augLag(newdiff_L7, diff_f7, diff_c7, x7_0, 100,1)
println("Aug Lag with exact Newton produced: ", xstar, " in ", k, "iterations.");

passed iteration 2 with x =[0.0,1.8083056690029229], c(x) = 0.26996939254810837
passed iteration 3 with x =[0.0,1.7547985051328368], c(x) = 0.07931779361643798
passed iteration 4 with x =[0.0,1.7393494212074738], c(x) = 0.0253364090547743
passed iteration 5 with x =[0.0,1.7344135514322154], c(x) = 0.008190367391710751
passed iteration 6 with x =[0.0,1.7328178995660877], c(x) = 0.002657873056628013
passed iteration 7 with x =[0.0,1.73230008592508], c(x) = 0.0008635876960392252
passed iteration 8 with x =[0.0,1.732131839022089], c(x) = 0.00028070775404387405
passed iteration 9 with x =[0.0,1.732077150588756], c(x) = 9.125559166456298e-5
passed iteration 10 with x =[0.0,1.7320593788171295], c(x) = 2.9691748380855643e-5
passed iteration 11 with x =[0.0,1.7320535894731373], c(x) = 9.636806779234064e-6
passed iteration 12 with x =[0.0,1.7320517105740263], c(x) = 3.128102409988287e-6
passed iteration 13 with x =[0.0,1.732051100511158], c(x) = 1.0147819136108183e-6
passed iteration 14 with x =

In [41]:
(xstar,k)=augLag(newdiff_L9, diff_f9, diff_c9, x9_0, 100,3)
println("Aug Lag with BFGS produced: ", xstar, " in ", k , " iterations.");
(xstar,k)=augLag(newdiff_L9, diff_f9, diff_c9, x9_0, 100,1)
println("Aug Lag with exact Newton produced: ", xstar, " in ", k, "iterations.");

passed iteration 2 with x =[-3.400752632299028,-3.4659451486326796], c(x) = -3.205175083298073
passed iteration 3 with x =[-3.0409470182607836,-3.945538087950912], c(x) = -0.3271738091903984
passed iteration 4 with x =[-3.00000950114947,-3.9999875265035754], c(x) = -7.542508715374652e-5
passed iteration 5 with x =[-3.000000000058943,-3.9999999999214553], c(x) = -4.714060253263597e-10
Aug Lag with BFGS produced: [-3.000000000058943,-3.9999999999214553] in 5 iterations.
passed iteration 2 with x =[-3.4001173411458305,-3.466510211805559], c(x) = -3.200938729166646
passed iteration 3 with x =[-3.0408967761875303,-3.9454709650832553], c(x) = -0.32717420950035603
passed iteration 4 with x =[-3.0000093781555797,-3.999987495792561], c(x) = -7.502524463554039e-5
passed iteration 5 with x =[-3.000000000000493,-3.999999999999342], c(x) = -3.9470648971473565e-12
Aug Lag with exact Newton produced: [-3.000000000000493,-3.999999999999342] in 5iterations.
